In [52]:
import evol_dynamics

In [53]:
import random

In [54]:
import numpy as np

In [55]:
import tqdm

In [56]:
def simulation(
    N,
    delta,
    beta,
    mutation,
    number_of_steps,
    payoffs,
    mode,
    filename,
    seed=10,
    starting_resident=(0, 0, 0),
):

    data = [0, *payoffs, N, delta, beta, mutation, mode, *starting_resident, 0]
    
    with open(filename, "w") as textfile:
        textfile.write(",".join([str(elem) for elem in data]) + "\n")
    textfile.close()
    
    population = {starting_resident: N}
    random.seed(seed)


    for i in tqdm.tqdm_notebook(range(1, number_of_steps + 1)):
        
        if random.random() < mutation:
            to_mutate = random.sample(population.keys(), 1)
            population[to_mutate[0]] -= 1
            population[tuple(random.random() for _ in range(3))] = 1
            
        else:
        
            resident, mutant = random.sample(population.keys(), 2)
        
            if mode == "expected":

                resident_payoff, mutant_payoff = payoffs_expected(resident, mutant, population, payoffs, delta, N)
                
            if mode == "last_round":

                resident_payoff, mutant_payoff = payoffs_last_round(resident, mutant, population, payoffs, delta)
                
            fermi = 1 / (
                1
                + np.exp(
                    float(
                        -beta
                        * (resident_payoff - mutant_payoff)
                    )
                )
            )
                
            if random.random() < fermi:
                population[resident] -= 1
                population[mutant] += 1
            else:
                population[mutant] -= 1
                population[resident] += 1

            for member in population.items():

                data = [i, *payoffs, N, delta, beta, mutation, mode, *member[0], member[1]]

                with open(filename, "a") as textfile:
                    textfile.write(",".join([str(elem) for elem in data]) + "\n")
                textfile.close()

        population = {key:val for key, val in population.items() if val != 0}
    return population


In [189]:
(1, 0, 3) * 2

(1, 0, 3, 1, 0, 3)

In [191]:
[key for key in population.keys() for _ in range(population[key])]

[(1, 0, 1), (1, 0, 1), (1, 0, 1), (1, 1, 1), (0, 1, 1)]

In [142]:
def payoffs_expected(resident, mutant, population, payoffs, delta, N):

    resident_payoff = sum(
        [
            evol_dynamics.expected_distribution_last_round(
                resident, opponent, delta
            )
            @ payoffs
            * (population[opponent] - int(resident == opponent))
            for opponent in population.keys()
        ]
    ) / (N - 1)

    mutant_payoff = sum(
        [
            evol_dynamics.expected_distribution_last_round(
                mutant, opponent, delta
            )
            @ payoffs
            * (population[opponent] - int(mutant == opponent))
            for opponent in population.keys()
        ]
    ) / (N - 1)

    return resident_payoff, mutant_payoff

In [164]:
import copy

In [167]:
def payoffs_last_round(resident, mutant, population, payoffs, delta):
    
    population_cp = copy.deepcopy(population)
    
    opponent = random.sample(population_cp.keys(), 1)[0]
    population_cp[opponent] -= 1

    resident_payoff = evol_dynamics.expected_distribution_last_round(resident, opponent, delta) @ payoffs

    opponent = random.sample(population_cp.keys(), 1)[0]
    mutant_payoff = evol_dynamics.expected_distribution_last_round(mutant, opponent, delta) @ payoffs


    return resident_payoff, mutant_payoff



In [168]:
payoffs = np.array([2, -1, 3, 0])

In [124]:
resident = (1, 0, 1)
mutant = (1, 1, 1)

In [125]:
evol_dynamics.expected_distribution_last_round(resident, resident, delta=0.999) @ payoffs

1.0005002501250768

In [126]:
evol_dynamics.expected_distribution_last_round(mutant, mutant, delta=0.999) @ payoffs

2.0

In [127]:
evol_dynamics.expected_distribution_last_round(resident, mutant, delta=0.999) @ payoffs

2.9989999999999997

In [137]:
evol_dynamics.expected_distribution_last_round(mutant, resident, delta=0.999) @ payoffs

-0.997

In [128]:
other = (0, 1, 1)

In [129]:
evol_dynamics.expected_distribution_last_round(other, resident, delta=0.999) @ payoffs

-0.993003

In [130]:
evol_dynamics.expected_distribution_last_round(resident, other, delta=0.999) @ payoffs

2.9950010000000002

In [131]:
evol_dynamics.expected_distribution_last_round(other, mutant, delta=0.999) @ payoffs

2.001

In [132]:
evol_dynamics.expected_distribution_last_round(mutant, other, delta=0.999) @ payoffs

1.9969999999999999

In [133]:
population = {resident: 10, mutant: 5, other: 2}

In [143]:
payoffs_expected(resident, mutant, population, payoffs, 0.999, 17)

(1.8743440156953557, 0.12649999999999995)

In [144]:
(9 * 1.0005 + 5 * 2.99 + 2 * 2.995) / 16

1.8715312500000003

In [147]:
(10 * -0.997 + 4 * 2 + 2 * 1.996) / 16

0.12637499999999996

In [91]:
delta = 0.999

In [162]:
payoffs_last_round(resident, mutant, population, payoffs, delta)

(1.0005002501250768, 1.9969999999999999)

In [95]:
population.keys()

dict_keys([(1, 0, 1), (1, 1, 1), (0, 1, 1)])

In [173]:
population = {resident: 3, mutant: 1, other: 1}

In [176]:
payoffs_last_round(resident, mutant, population, payoffs, 0.99)

(2.9500999999999995, -0.97)

In [151]:
payoffs_expected(resident, mutant, population, payoffs, 0.99)

TypeError: payoffs_expected() missing 1 required positional argument: 'N'

In [163]:
np.linspace(0.9, 0.95, 10)

array([0.9       , 0.90555556, 0.91111111, 0.91666667, 0.92222222,
       0.92777778, 0.93333333, 0.93888889, 0.94444444, 0.95      ])